# Bruno Di Sanzo, legajo 62497

In [1]:
!pip uninstall tensorflow -y

Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Successfully uninstalled tensorflow-2.17.0


In [2]:
!pip install tensorflow==2.8

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 38.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.7.2
    Uninstalling tensorboard-data-server-0.7.2:
      Successfully uninstalled tensorboard-data-server-0.7.2
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: goo

In [3]:
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta.

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [4]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output

--2024-09-02 13:05:01--  https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223725 (218K) [image/jpeg]
Saving to: ‘La_noche_estrellada1.jpg’

La_noche_estrellada 100%[===================>] 218.48K  --.-KB/s    in 0.04s   

2024-09-02 13:05:01 (4.94 MB/s) - ‘La_noche_estrellada1.jpg’ saved [223725/223725]

--2024-09-02 13:05:01--  https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153015 (149K) [ima

In [5]:
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
import tensorflow as tf
from pathlib import Path

In [6]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

base_image_path = Path("/content/messi-world-cup.jpg")
style_reference_image_path = Path("/content/cubismo.jpg")
result_prefix = Path("/content/output")
iterations = 100

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta:

**style_weight**: es el peso que se le da a la función de costo relacionada al estilo. Esta función de costo compara la Gram matrix (matriz de correlación entre las distintas acrtivaciones) de la imagen "base" de estilo con la imagen a generar.

**content_weight**: es el peso que se le asigna a la función de costo relacionada al contenido. Esta función de costo compara las activaciones de una capa respecto a la imagen "base" y a la imagen a generar.

**total_variation_weight**: es el peso que se le asigna a la función de costo correspondiente a la "variación" de la imagen a generar. No se encuentra en el paper. En principio, parecería que mide cuánta diferencia hay localmente debido a un shifteo en alguna de las 4 direcciones (local porque se mueve sólo 1 unidad). A mayor diferencia haya entre píxeles cercanos, mayor dará este error. Parecería tener que ver, a priori, con qué tan suave queda la imagen. Determina qué tan ruidosa queda la imagen.

Estos pesos asignados a cada función de costo implican cuánto efecto tendrán en la función de costo total, que combina a las 3 funciones de costo.

In [7]:
total_variation_weight = 0.1
style_weight = 10
content_weight = 1

In [8]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

Respuesta:

La función es para preprocesar la imagen, leyéndola desde el path. Primero, carga la imagen a las dimensiones indicadas (lo fuerza a tener 400 filas y no perder el ratio).

Luego, lo convierte en un array y le agrega una dimensión. Esto es para que tenga la dimensión que entra a la Red, que no es otra cosa que la dimensión que indica el batch_size.

El preprocess_input de Vgg16, según la documentación, cambia el formato RGB a BGR, y después hace un zero-mean por cada canal de color: cada canal es centrado (normalizado en la media) respecto a la media de los valores de cada canal utilizados en el dataset de ImageNet (pero no las escala).

El preprocess_input es para que las imágenes de entrada tengan el mismo formato que las imágenes utilizadas para el entrenamiento y así la red funcione de manera óptima.

In [9]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Respuesta:

La celda siguiente es la "inversa" de la celda anterior. Dada una imagen que tiene el formato correspondiente a la salida de la celda anterior, le quita la dimensión extra para que sólo tenga Height, Width y Channels; y luego vuelve a centrar los valores de los píxeles de los canales, sumándole los valores correspondientes a las medias de los mismos usando el dataset de ImageNet.

Luego, convierte de BGR a RGB, se asegura de que los valores queden en el rango [0,255], y los castea a uint8, para que quede con el formato habitual de las imágenes.

In [10]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [11]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

In [12]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [13]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [14]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

80150528/80134624 [==============================] - 0s 0us/step
Model loaded.


# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram?¿Para qué se usa?

La matriz de Gram es una matriz que calcula la correlación entre los features maps de una capa. Tiene tamaño NlxNl, donde Nl es la cantidad de filtros de la capa correspondiente, que es justamente igual a la cantidad de feature maps. La correlación se calcula haciendo un producto escalar entre los vectores que representan a cada feature map entre sí. Es decir, en la posición (i,j) se encuentra la suma del producto elemento a elemento del feature map i con el feature map j.

Según el paper, la matriz de Gram contiene la información sobre el "estilo" de la imagen, por lo que se la usa para comparar una matriz de gram de la imagen "base" de estilo con la matriz de Gram de la imagen a generar, de manera tal que a una mayor similitud, la imagen a generar tendrá un estilo más similar.

- ¿Por qué se permutan las dimensiones de x?

Se permutan las dimensiones de x, ya que la última dimensión es igual a la cantidad de canales (o filtros), y la idea es que la multiplicación se realice entre los distintos feature maps. Al hacer este cambio del orden de las dimensiones, se logra que las multiplicaciones se hagan en las otras dimensiones, y que la salida sea de tamaño NlxNl.

Es más fácil verlo si se piensa en que, al cambiar las dimensiones, se forma un tensor de tamaño (Nl, H, W), que al realizar el producto punto se lo puede pensar como un producto de matrices: (Nl, (HxW)) x ((HxW), Nl); donde cada elemento de esta "matriz" es una matriz (un feature map), de manera que el producto se realiza entre esos feature maps (acá, el producto el element-wise, tratándolo como si fuera un sólo vector, y luego lo suma. Igual que la cuenta que aparece en el paper).

In [15]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

Rta:

**style_loss**: es la función de costo que mide la "diferencia" entre los estilos de la imagen "base" de estilo y la imagen a generar. En particular, calcula el distancia Mean Square entre las matrices de Gram de la imagen "base" y la que se generará (es decir, hace diferencia cuadrática elemento a elemento, y luego suma todos los valores, tal como hace el paper). Un detalle, en el paper channels es igual a la cantidad de filtros de la capa en la que se están calculando las matrices de Gram, que no necesariamente es 3. De todas maneras, es sólo un factor de escala.

**content_loss**:  es la función de costo que mide la "diferencia" entre los contendos de la imagen "base" de contenido y la imagen a generar. En particular, calcula es Square error entre los feature maps generados por la imagen original y la imagen a generar (nuevamente, calcula el error cuatrático elemento a elemento, y luego suma todos los valores). En el paper se le agrega un factor de escala de 1/2 que acá no está.

**total_variation_loss**: es una función de costo que no aparece en el paper. Calcula la diferencia cuadrática entre el tensor (imagen a generar) y un shifteo del mismo en 1 unidad en sentido vertical (se elimina la última columna, y se considera el tensor primero sin la primera fila y restándole el tensor sin la última fila, de ahí el shifteo), luego en sentido horizontal (se elimina la última fila, y se considera el tensor primero sin la primera columna y restándole el tensor sin la última fila, de ahí el otro shifteo), suma estas diferencias cuadráticas elemento a elemento (queda un tensor donde las dimensiones de alto y ancho son 1 menos que antes); y luego eleva elemento a elemento a la 1.25 y suma todos estos valores. En principio, parecería que mide cuánta diferencia hay localmente debido a un shifteo en alguna de las 4 direcciones (local porque se mueve sólo 1 unidad). A mayor diferencia haya entre píxeles cercanos, mayor dará este error. Parecería tener que ver, a priori, con qué tan suave queda la imagen.

Finalmente, se suman las 3 funciones de costo con sus respectivos pesos. Para la **style_loss**, se calculan varias funciones de costo, cada una para una salida de una capa distinta, y luego se la normaliza por la cantidad de layers que se usaron.

En este caso, para el "contenido", se usa la salida de la segunda convolución del bloque 5 y para el "estilo", la primera convolución de los bloques 1, 2, 3, 4 y 5.

In [16]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [17]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [18]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [19]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [20]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:

El paper dice de usar gradient descent para optimizar la función de costo.

En cambio, **fmin_l_bfgs_b** implementa el algoritmo Limited-memory BFGS. Este algoritmo es un cuasi método de Newton, en el cual se tiene en cuenta la matriz hessiana (segunda derivadas) para hallar la dirección y magunitud en la cual moverse dentro de la función de costo. El método, en particular, calcula una estimación de la matriz Hessiana, la cual va actualizando por cada paso.

In [21]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [22]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [24]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(10):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d.png' % i)
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 64072147000.0
Image saved as /content/output/output_at_iteration_0.png
Iteration 0 completed in 604s
Start of iteration 1
Current loss value: 37225247000.0
Image saved as /content/output/output_at_iteration_1.png
Iteration 1 completed in 591s
Start of iteration 2
Current loss value: 26837080000.0
Image saved as /content/output/output_at_iteration_2.png
Iteration 2 completed in 593s
Start of iteration 3
Current loss value: 21834205000.0
Image saved as /content/output/output_at_iteration_3.png
Iteration 3 completed in 588s
Start of iteration 4
Current loss value: 19020694000.0
Image saved as /content/output/output_at_iteration_4.png
Iteration 4 completed in 579s
Start of iteration 5
Current loss value: 17021680000.0
Image saved as /content/output/output_at_iteration_5.png
Iteration 5 completed in 583s
Start of iteration 6
Current loss value: 15834366000.0
Image saved as /content/output/output_at_iteration_6.png
Iteration 6 completed in 587s
Start 

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Respuesta:

Cabe aclarar que se hicieron pocas iteraciones, porque tardaba 5 minutos por iteración y además se terminaba el tiempo de GPU del Google Colab.

total_variation_weight = 0.1,
style_weight = 1,
content_weight = 10

A mayor valor el peso de la función de costo del contenido, mayor es el componente correspondiente al contenido y menos al estilo.

total_variation_weight = 0.1,
style_weight = 100,
content_weight = 1

De la misma manera, a mayor peso de la función de costo correspondiente al estilo, más se nota el estilo de la imagen y menos el contido (se pierde la forma original de los objetos).


total_variation_weight = 1,
style_weight = 10,
content_weight = 1

Por último, a mayor peso de "variation", la imagen se ve más suave, levemente más "borrosa". Es decir, con menos variación entre píxeles.

# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

Respuesta:

Ver imágenes adjuntadas
